<a href="https://colab.research.google.com/github/nationalarchives/I-CeM-workshop-2022/blob/main/Colabs/01_Loading_and_viewing_I_CeM_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook outline

This notebook accompanies the workshop "Applying computational methods to census data from I-CeM" run by The National Archives on 28th June 2022.

The notebook demonstrates opening a data extract from the I-CeM platform, exploring the structure, viewing records, and creating basic summaries.

The notebook does not contain any data, so first we must upload a file. Follow the instructions provided in the Github repository (https://github.com/nationalarchives/I-CeM-workshop-2022/blob/main/README.md) to upload the data which was extracted from I-CeM earlier in the session. The notebook is intended to work with the full extract which is obtained by selecting "Download all variables (including raw textual variables)" at the Download Options stage.

# Initial setup

The tutorial will use the tidyverse library which contains a number of packages for working with data and implementing Tidy Data principles (see https://vita.had.co.nz/papers/tidy-data.pdf). The primary packages we will use are dplyr which is used for data manipulation, and ggplot2 for visualisation. The repr package is also used to change the size of visualisations.

In addition a couple of extra functions are created which will be described later in the notebook.

For now click in the brackets to the left of "3 cells hidden" below.

# Load libraries and useful functions

In [ ]:
library(tidyverse)
library(repr)

In [ ]:
source('https://raw.githubusercontent.com/nationalarchives/I-CeM-workshop-2022/main/common_r_functions.R')
source('https://raw.githubusercontent.com/nationalarchives/I-CeM-workshop-2022/main/table_creation_functions.R')

In [ ]:
variable_file <- 'https://raw.githubusercontent.com/nationalarchives/I-CeM-workshop-2022/main/Data/census%20variable%20descriptions.csv'
variables <- read.csv(variable_file)
occode_file <- 'https://raw.githubusercontent.com/nationalarchives/I-CeM-workshop-2022/main/Data/occode_lookup.tsv'
occode_lookup <- read.csv(occode_file, sep = '\t')

# Loading the data file

Having uploaded the data we can load it into a data frame - a structure for working with tabular data. There is unfortunately an issue with the raw data having two too few headings in the first row. This prevents the file being loaded without an intervention. We can do this programmatically by reading the file, appending two column names to the first row, and writing the results to a new file. This also provides an opportunity to rename the file to something more meaningful along the way. The code has been hidden but can be revealed by clicking on 'Show code' below if you're particularly interested.

Start this section by changing the value of the SOURCE_DATA_FILE variable in the cell below. In Colab you can get the name of this file by right clicking on the file in the 'Files' pane and selecting 'Copy path'. Then paste the copied value in the cell below, where it says 'Insert text here'. (Note: this input form does not work well when typing a value in, so it is better to paste - also reduces likelihood of typing errors with a long filename)

The result should look something like this (your filename will be different):

'/content/b72b4efe-e613-40f5-8ac6-6fe222f686c3.zip'


If the previous step was followed correctly the cell will run without error and the data will be loaded into a table called 'source_data'
(technically the data structure we're loading into is called a tibble in the world of dplyr, but table is a more commonly understood term so we will use that in this tutorial).

It will also print out the number of rows and columns in the data frame.

In [ ]:
#@title ...Enter source data file name { run: "auto", vertical-output: true, form-width: "50%", display-mode: "form" }

SOURCE_DATA_FILE = "" #@param {type:"string"}

SOURCE_DATA_FILE

temp_file <- 'temporary_file.txt'
fix_header(SOURCE_DATA_FILE, temp_file)
source_data <-  read_delim(temp_file, delim = '|', col_names = TRUE, show_col_types = TRUE)
paste(c("Rows:",nrow(source_data), "Columns:", ncol(source_data)), sep = " ")


### Problems with the data

In fact for the Rutland 1851 download there are some issues with the data and we get a warning message telling us that there were issues parsing the data. This comes down to this particular library inferring data types from the column values. In this case it has inferred that the H_Mar (marital status of head of household) column is a number (which according to the I-CeM guide is correct) but found 7 entries containing a '.' character.

The problems() function shows this and the colnames function is used to confirm that the problematic column (89) is indeed H_Mar.

The I-CeM guide describes the logic we could use to infer the most likely value for H_Mar but it involves analysis of the household which is beyond the scope of this tutorial. For now they will remain as NA, the default set for non-numerical values.

In [ ]:
problems(source_data)
colnames(source_data)[89]
source_data[15518, 89]

## View the structure

Having loaded the data we can now view the list of columns in the data.

In [ ]:
colnames(source_data)

Unfortunately the column names are not always obviously interpretable. While the meanings of Country, Parish, Population are clear, columns such as RC, RD and CFU may require further explanation.



## Load variable datasheet

The I-CeM project does provide a datasheet which gives a long label for each field. Census downloads are standardised so that each download contains the same columns, but not all variables were captured in any given year. So the datasheet also provides a handy lookup for which fields are populated in each year. The datasheet will be loaded into two tables:



*   column_labels: maps field name to long label; two variables indicate whether field is coded, and in the anonymised dataset.
*   columns_by_census: for each variable (VariableName column) and each census (census_name column) the has_variable column indicates whether that variable is in that census.

After loading the data the first three rows of each table will be output, column_labels first, columns_by_census second.


A full explanation of how the tables are created is included in the hidden Explanation section which follows.

In [ ]:
#@title Click the arrow to load the data
variable_file <- 'https://raw.githubusercontent.com/nationalarchives/I-CeM-workshop-2022/main/Data/census%20variable%20descriptions.csv'
variables <- read.csv(variable_file)

column_labels <- variables %>%
 select("VariableName", "Label","Coded","AnonymisedVersion") %>%
 mutate(Coded = (Coded == "x"), AnonymisedVersion = (AnonymisedVersion == "x"))

columns_by_census <- variables %>%
 select("VariableName", EW51:SCOT01) %>%
 pivot_longer(EW51:SCOT01, names_to = "census_name", values_to = "has_variable") %>%
  mutate(has_variable = (has_variable == "x"))

head(column_labels,3)
head(columns_by_census,3)

### Explanation (expand to read)

First we load the data file into a variable called variables. The data file is hosted on Github and is in csv (Comma Separated Values) format. The first row is a header row with column names.

In [ ]:
variable_file <- 'https://raw.githubusercontent.com/nationalarchives/I-CeM-workshop-2022/main/Data/census%20variable%20descriptions.csv'
variables <- read.csv(variable_file, header = TRUE)


The column_labels table contains the mapping of field name to long label, as well as variables indicating whether they are Coded and/or included in the Anonymised dataset (which we are using). The latter two columns are converted from 'x'/blank to TRUE/FALSE values using the mutate function. Within the mutate function 'CODED = (CODED == "x")' means that the CODED field will be set to the value of (CODED == "x"). This is a boolean test which will return TRUE if the original CODED value was "x" and FALSE otherwise. The Anonymised.version field works the same way.


In [ ]:
column_labels <- variables %>%
 select("VariableName", "Label","Coded","AnonymisedVersion") %>%
 mutate(Coded = (Coded == "x"), AnonymisedVersion = (AnonymisedVersion == "x"))

The columns_by_census table follows Tidy Data principles to convert from having a column per census, to a row per variable per census. The pivot_longer function converts the columns from EW51 to EW11 (the England and Wales censuses) and from SCOT51 to SCOT01 (Scotland's censuses) to two columns called census_name (the original column name, e.g. E.W51) and has_variable (the value in the original column). Again the latter is converted to a TRUE/FALSE value.

In [ ]:
columns_by_census <- variables %>%
 select("VariableName", EW51:SCOT01) %>%
 pivot_longer(EW51:SCOT01, names_to = "census_name", values_to = "has_variable") %>%
  mutate(has_variable = (has_variable == "x"))

## 1851 variables

We can now use these two tables to trim down the original data table to only include columns which are populated in the 1851 England and Wales census and which appear in the anonymised extract. A new table  called census_data is created and will be used in the rest of the tutorial. The resulting list of columns in this table and their long label are printed at the end.

The code is hidden but a full explanation of it is available for the curious in the hidden section following the code (called Explanation).

In [ ]:
#@title Click arrow to run { vertical-output: true }
selected_columns <- columns_by_census %>%
 filter(census_name == 'EW51' & has_variable) %>%
  inner_join(column_labels, by = 'VariableName') %>%
 filter(AnonymisedVersion) %>%
 select("VariableName", "Label")

census_data <- source_data %>% select(str_trim(selected_columns$VariableName))

selected_columns %>% print(n=100)

### Explanation (expand to read)



The value in the census_name column which was created when we tidied the data is 'EW51' (the ampersands in the original data were removed), and the has_variable column is TRUE if that column is populated. We include both of these expressions within dplyr's filter function on line 2.

The inner_join function joins the columns_by_census table to the column_labels table so that we can get the full column labels and also identify which are Coded and available in the Anonymised download.

Finally we use dplyr's select function to create a trimmed down version of the source data using only the select columns. Some of the values in the VariableName column appear to have trailing space, so they are removed using the str_trim function.

The resulting table is output at the end. It is called census_data and will be used in the remainder of this tutorial.

In [ ]:
selected_columns <- columns_by_census %>%
 filter(census_name == 'EW51' & has_variable) %>%
  inner_join(column_labels, by = 'VariableName') %>%
 filter(AnonymisedVersion) %>%
 select("VariableName", "Label")
 
census_data <- source_data %>% select(str_trim(selected_columns$VariableName))

selected_columns %>% print(n=100)

# Exploring the data



## Viewing the data

The obvious starting point when first exploring the data is to look at some records. This can be simply achieved using the head function but unfortunately it doesn't output many of the columns due to space restrictions. So we have to set the width option and use the print function to produce a full width output. The value inside the head function can be changed to view more, or fewer, records.

In [ ]:
options(width=1000)
census_data %>% head(5) %>% print()

Instead of the top 5 rows it can be more informative to view a random sample instead. In this case we switch the head function for the sample_n function.

In [ ]:
census_data %>% sample_n(5) %>% print()

## Data summary

We can use the summary function to gain an overview of the data. Most of the columns are text, which summary doesn't do anything particularly interesting with (see the output under RegDist for an example), so we have chosen a selection of numerical columns to make the output more digestible.

The column summaries highlight some interesting features worthy of further investigation. We can see the range of populations across the districts from a minimum of 7 to a maximum of 2092, and that there appears to be a roughly 50:50 split between male and female populations. The Rela field has an extreme value of 9999 which makes the mean much higher than the 3rd Quartile value. This is most likely a default value whose meaning should be looked up. The RelInf column is similar. The HeadInf column looks like it is a boolean value (i.e. 0s and 1s). The H_Age column (age of head of household) has very high values for the quartiles, median and mean. Again the documentation and further analysis can be used to understand its contents.

In [ ]:
options(width=200)
summary(census_data[c("Year","RecID","RegDist","Population","MalePop","FemalePop","Rela","RelInf","HeadInf","H_Age")])

## Sampling rows

For the purposes of the next section we will use a subset of columns to make the output easier to digest. We will use the select function to achieve this rather than creating another trimmed down table.

The next statement defines the columns and prints out a list with labels.

In [ ]:
output_columns <- c("RecID", "RegDist", "Parish", "Population", "Censusref", "H", "PID", "Sex", "Age", "Cond", "Relat", "Occ", "Bpstring", "BpCmty")
output_columns <- all_of(output_columns)
column_labels %>% filter(VariableName %in% output_columns) %>% select(VariableName, Label, Coded)

We will again use the sample_n function, with a parameter of 1, to select a random record. The select function is used to choose the columns defined above.
Every time you run the cell below a new row will be produced.

Try running the cell a few times.

In [ ]:
census_data %>% sample_n(1) %>% select(all_of(output_columns))

Instead of looking at a single record, let's look at the record for an entire household. The following query will use the Censusref and H (House Identifier) values from a new sample row and return every record with the same values.

Again try it a few times.

In [ ]:
sample_row <- census_data %>% sample_n(1) %>% select(all_of(output_columns))

census_data %>% filter(Censusref == sample_row$Censusref[1] & H == sample_row$H[1]) %>% select(all_of(output_columns))

## Data profiling

Looking at individual rows is a good starting point for getting to know a dataset, but it is often better to look at the data in aggregate to understand its distribution and identify patterns. This process is sometimes termed Data Profiling.

Two functions which characterise the format of a string have been created to aid profiling.



*   value_format: converts letters to 'a' or 'A' depending on case, and numeric characters to '9'. All other characters remain the same.
    Example: Bcde-3 becomes Aaaa-9
*   value_format_compress: similar to value_format but replaces runs of repeated characters ('a','A', or '9') with an asterix.
    Example: Bcde-3 becomes Aa*-9

You can try out the functions with different strings in the next two cells. Just change the values in quotation marks.


In [ ]:
value_format("Bcde-3")

In [ ]:
value_format_compress("Bcde-3")
value_format_compress("abcdxyz 1289ABCXYZ-")

### Counting values

An obvious starting point is to find the most commonly occurring values in a column. This can be achieved using the group_by and summarise functions. Select a field from the dropdown list in the form below, then select a function to use:


*   Value: group by the field value
*   ValueFormat: group by the format of the value using the value_format function
*   ShortFormat: group by the shortened version of the format using the value_format_compress function


 Use the slider to adjust the number of values it returns. The query will run automatically when changing the values in the form.

In [ ]:
#@title Select a field to summarise { run: "auto", vertical-output: true }
fieldname = 'Occ' #@param ["Censusref", "PID", "Occ", "Sex", "Parish", "Cond", "Relat", "Age"]
aggregation_function = 'Value' #@param ["Value","ValueFormat","ShortFormat"]
records = 12 #@param {type:"slider", min:1, max:30, step:1}

Value <- function(v) { return(v) }
ValueFormat <- function(v) { return(value_format(v)) }
ShortFormat <- function(v) { return(value_format_compress(v)) }

function_to_use <- get(aggregation_function)

census_data %>%
 select(aggregate_column = all_of(fieldname)) %>% 
 mutate(aggregate_column = lapply(aggregate_column, function_to_use))  %>%
   group_by(aggregate_column) %>% summarise(count = n()) %>%
    arrange(-count) %>%
    head(records)

# Visualisations

While numerical summaries are interesting, data really comes alive when we visualise it.

In the following section you can select a field name and create a visualisation showing counts of records by value. Optionally the visualisations can be partitioned by a second variable (select Blank to not do this). The slider will set how many values will be returned - it will return the top N rows by size (where N is the value set by the slider).

In [ ]:
#@title Select a field to summarise { run: "auto", vertical-output: true }
FieldName = 'Parish' #@param ["Censusref", "PID", "Occ", "Sex", "Parish", "Cond", "Relat", "Age"]
PartitionField = 'is_child' #@param ["BlankCol", "Sex", "is_child"]
records = 15 #@param {type:"slider", min:5, max:30, step:1}


options(repr.plot.width=20, repr.plot.height=8, warn = -1)
data_summary <- census_data %>%
    mutate(Age = as.numeric(Age), BlankCol = '') %>%
    filter(!is.na(Age)) %>%
    mutate(is_child = (Age < 18)) %>%
    select(aggregate_column = all_of(FieldName), fill_column = all_of(PartitionField)) %>%
    group_by(aggregate_column, fill_column) %>% summarise(count = n())

aggregate_col_summary <- data_summary %>%
    group_by(aggregate_column) %>%
    summarise(count_sum = sum(count)) %>%
    arrange(-count_sum) %>% head(records)

data_summary %>% 
    inner_join(aggregate_col_summary, by = 'aggregate_column') %>%
        ggplot(aes(x = reorder(aggregate_column, -count), y = count, fill = fill_column)) +
            geom_bar(stat="identity") + coord_flip() + facet_wrap(vars(fill_column)) + xlab(FieldName) +
            theme(axis.text.x = element_text(size = 15, angle = 55, vjust = 1.0, hjust=1.0))

# Summary

In this notebook we have:



*   Loaded an I-CeM data file into a table for analysis
*   Identified issues in both the header row and the format of one of the columns
*   Used metadata to identify and select which columns are relevant to a particular census
*   Browsed data rows and created aggregated summaries to better understand the data
*   Created simple bar chart visualisations of key columns

